In [3]:
import cupy
import time

iter_times = 100
n_matrices = 200

matrices = []
the_sum = None

for_loop_exec_times = []
cupy_sum_exec_times = []

for i in range(n_matrices):
  matrices.append(cupy.random.normal(size=(600,600)))


for j in range(iter_times):
  the_sum = None
  start_time = time.time()
  for i in matrices:
    the_sum = i if type(the_sum) == type(None) else the_sum + i
  end_time = time.time()
  for_loop_exec_times.append(end_time - start_time)

for j in range(iter_times):
  the_sum = None
  start_time = time.time()
  cupy_matrices = cupy.array(matrices)
  the_sum = cupy.sum(cupy_matrices)
  end_time = time.time()
  cupy_sum_exec_times.append(end_time - start_time)

for_loop_exec_times_avg = cupy.average(cupy.array(for_loop_exec_times))
for_loop_exec_times_std = cupy.std(cupy.array(for_loop_exec_times))

cupy_sum_exec_times_avg = cupy.average(cupy.array(cupy_sum_exec_times))
cupy_sum_exec_times_std = cupy.std(cupy.array(cupy_sum_exec_times))


print('for loop exec time average:', for_loop_exec_times_avg)
print('for loop exec time std:', for_loop_exec_times_std)
print('\ncupy sum exec time average:', cupy_sum_exec_times_avg)
print('cupy sum exec time std:', cupy_sum_exec_times_std)


for loop exec time average: 0.03706991195678711
for loop exec time std: 0.03415125297370427

cupy sum exec time average: 0.04074386835098267
cupy sum exec time std: 0.004220867425526529
